In [1]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import datetime as datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from API_keys import (consumer_key,
                    consumer_secret,
                    access_token,
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
target_users = ["@BBCWorld", "@CBSNews", "@CNN", "@FoxNews", "@nytimes"]

In [6]:
# Variables for holding sentiments
source_account = []
text = []
date = []
tweets_ago = []
compound_list = []
positive_list = []
negative_list = []
neutral_list = []


# Looping through all target users
for user in target_users:
     # Variable for holding the oldest tweet
    oldest_tweet = None

    tweet_count = 0
    # Get the last 100 tweets
    for x in range(5):
        # get all tweets from the home feed
        public_tweets = api.user_timeline(user, count=100, result_type="recent", max_id=oldest_tweet)
        #loop through all tweets
        for tweet in public_tweets:
            # keep adding the tweet_count
            tweet_count +=1
            #append values to empty lists
            source_account.append(user)
            text.append(tweet['text'])
            date.append(tweet['created_at'])
            tweets_ago.append(tweet_count)
            
            # Run the Vader Analysis on each tweet
            compound = analyzer.polarity_scores(tweet['text'])['compound']
            pos = analyzer.polarity_scores(tweet['text'])['pos']
            neg = analyzer.polarity_scores(tweet['text'])['neg']
            neu = analyzer.polarity_scores(tweet['text'])['neu']
            
            # append the values to the lists 
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)

In [8]:
# Create a dictionary for the DataFrame
tweet_sentiment = {
    "Source Account": source_account,
    "Tweet": text,
    "Date": date,
    "Tweets Ago": tweets_ago,
    "Compound Score": compound_list,
    "Positive Score": positive_list,
    "Negative Score": negative_list,
    "Neutral Score":  neutral_list,
    
}

tweets_df = pd.DataFrame(tweet_sentiment, columns = ['Source Account',
                                                     'Tweet',
                                                     'Date',
                                                     'Tweets Ago',
                                                     'Compound Score',
                                                     'Positive Score',
                                                     'Negative Score',
                                                     'Neutral Score']).set_index("Source Account")
tweets_df.head()

,Tweet,Date,Tweets Ago,Compound Score,Positive Score,Negative Score,Neutral Score
Source Account,,,,,,,
@BBCWorld,Supreme Court: Why a fight over US abortion la...,Wed Jun 27 22:12:31 +0000 2018,1,0.1027,0.228,0.266,0.506
@BBCWorld,Five attempts to make toys more inclusive http...,Wed Jun 27 21:34:47 +0000 2018,2,0.0000,0.000,0.000,1.000
@BBCWorld,Asylum seekers in Ireland to be allowed to wor...,Wed Jun 27 21:22:33 +0000 2018,3,0.0000,0.000,0.000,1.000
@BBCWorld,Flamingo that escaped a zoo in 2005 spotted in...,Wed Jun 27 19:54:02 +0000 2018,4,0.0000,0.000,0.000,1.000
@BBCWorld,Antwon Rose: Pittsburgh officer charged with c...,Wed Jun 27 19:25:28 +0000 2018,5,-0.6369,0.000,0.426,0.574
